In [1]:
# An initial implementation, will be updated.

import numpy as np
import math



def calc_PDSOCt(BNPPt_est, Sf, Lo, LIGCELL, FIRE):
    """
    Calculate Plant-Derived SOC (PDSOCt) for year t.

    Parameters:
        BNPPt_est (float): Belowground Net Primary Production estimate (output of calc_BNPPest).
        Sf (float): Biomass at the end of the growing season (output of calc_Sf).
        Lo (float): Loss of biomass during the off-season (output of calc_Lo).
        LIGCELL (float): Lignin and cellulose content of livestock feed for year t (%).
        FIRE (float): Average number of fires per year (#/year).

    Returns:
        float: Plant-Derived SOC (PDSOCt) for year t.
    """
    PDSOCt = 0.45 * ((LIGCELL * (Sf - Lo / 2) * (1 - FIRE)) + (LIGCELL + 0.05) * BNPPt_est)
    return PDSOCt

def calc_DDSOCt(LIGCELL, Ddays, Cg, n, d, Lo):
    """
    Calculate dung-derived SOC (DDSOC) in year t.

    Parameters:
    - LIGCELL (float): Lignin and cellulose content of livestock feed for year t (%).
    - Ddays (int): Number of days in the grazing episode.
    - Cg (float): Daily consumption rate (g/animal/day). For the dormant season, the rate is assumed to be half this value.
    - n (int): Number of "pastures" per total area.
    - d (float): Stocking density (head/ha).
    - Lo (float): Loss of biomass during the off-season (g/m^2).

    Returns:
    - DDSOCt (float): Dung-derived soil organic carbon (gC/m^2/year).
    """
    DDSOCt = LIGCELL * 0.45 * 0.3 * ((Ddays * Cg * n * d * 10**(-4)) + Lo)
    return DDSOCt


def calc_SOCeq(PDSOCt, DDSOCt, SAND, RAIN, Gdays, lowSOC=False, c1=0.000358):
    """
    Calculate Equilibrium SOC (SOCeq).
    
    Parameters:
        PDSOCt (float): Output of calc_PDSOCt().
        DDSOCt (float): Output of calc_DDSOCt().
        SAND (float): Sand percentage in the top 30 cm soil.
        RAIN (float): Mean annual precipitation for the year (mm/year).
        Gdays (float): Total number of days in the growing season.
        lowSOC (bool): Whether lowSOC regression equation is used. Default is False.
        c1 (float): Coefficient for microbial respiration rate. Default is 0.000358.
    
    Returns:
        float: Equilibrium SOC (SOCeq).
    """
    WETDAYS = (c1 * RAIN - 0.025) * Gdays

    if lowSOC:
        SOCeq = ((PDSOCt + DDSOCt) / (WETDAYS * (0.7 + 0.3 * (SAND / 100)) * np.exp(-10.872))) ** (1 / 1.296)
    else:
        SOCeq = ((PDSOCt + DDSOCt + 0.579 * WETDAYS * (0.7 + 0.3 * (SAND / 100))) /
                 (c1 * WETDAYS * (0.7 + 0.3 * (SAND / 100)))) + 0.579

    return SOCeq


def calc_ANPPmax(RAIN, MAT, SAND):
    """
    Calculate maximum Above-ground Net Primary Production (ANPP).

    Parameters:
    - RAIN (float): Long-term mean annual precipitation (mm/year).
    - MAT (float): Long-term mean annual temperature (°C).
    - SAND (float): Sand percentage in the top 30 cm of soil (%).

    Returns:
    - ANPPmax (float): Maximum Above-ground Net Primary Production (g/m^2/year).
    """
    # Calculate the exponential suffix
    e_suffix = 12.039 + 0.718 * math.log(RAIN) - (25.18 / (0.00834 * (273.15 + MAT)))

    # Calculate ANPPmax
    ANPPmax = math.exp(e_suffix) * (1.33 - 0.0075 * SAND)
    return ANPPmax


def calc_ANPPest(Se, Sg, Sf, Sk, S0=None):
    """
    Calculate the parameter ANPPest, the total seasonal grass production for a given grazing management system,
    based on Ritchie (2020). Equivalent to Pg.

    Parameters:
    Se (float): Biomass at the start of the grazing episode (output of calc_Se).
    Sg (float): Biomass at the end of the grazing episode (output of calc_Sg).
    Sf (float): Biomass at the end of the growing season (output of calc_Sf).
    Sk (float): Steady state of biomass in the absence of grazing for a given location.
                This should ideally be measured directly using grazing exclosures.
    S0 (float, optional): Biomass condition prior to the growing season (at the end of the dry season),
                          mostly comprised of carbon stores in rhizomes. Default is 0.1 * Sk.

    Returns:
    float: The total seasonal grass production, equivalent to Pg.
    """
    if S0 is None:
        S0 = 0.1 * Sk

    Pg = (Se - S0) + (Sf - min(Sg, Se))
    return Pg


def calc_BNPPest(RAIN, MAT, ANPPt_est, Sk, S0=None, APCcorrection=False, DEPTH=30):
    """
    Calculate estimated Below-ground Net Primary Production (BNPP).

    Parameters:
    - RAIN (float): Long-term mean annual precipitation (mm/year).
    - MAT (float): Long-term mean annual temperature (°C).
    - ANPPt_est (float): Total seasonal grass production for the given grazing management system.
    - Sk (float): Steady-state biomass in the absence of grazing.
    - S0 (float, optional): Biomass condition prior to the growing season (g/m^2). Default is 0.1 * Sk.
    - APCcorrection (bool): If True, applies a correction factor for dominance of annual plants. Default is False.
    - DEPTH (float): Depth of soil sampling or estimation (cm). Default is 30 cm.

    Returns:
    - BNPPest (float): Estimated Below-ground Net Primary Production (g/m^2/year).
    """
    if S0 is None:
        S0 = 0.1 * Sk

    # Apply APC correction
    APC = 0.291 if APCcorrection else 1

    # Calculate BNPPest
    BNPPest = (0.602 * RAIN - 0.00038 * RAIN**2 + 5.88 * MAT) * (ANPPt_est / (Sk - S0)) * APC * (DEPTH / 40)
    return BNPPest




def calc_DMRESP(SOCt, lowSOC=False, c1=0.000358):
    """
    Calculate daily microbial respiration (DMRESP) in gC/m^2/year.

    Parameters:
    - SOCt (float): Soil organic carbon stocks in year t (gC/m^2).
    - lowSOC (bool): If True, use the low SOC regression equation (default: False).
    - c1 (float): Coefficient from the microbial respiration rate linear equation for high SOC (default: 0.000358).

    Returns:
    - DMRESP (float): Daily microbial respiration rate.
    """
    if lowSOC:
        DMRESP = math.exp(-10.872) * (SOCt ** 1.296)
    else:
        DMRESP = -0.579 + c1 * SOCt

    return DMRESP


def calc_MRESPt(RAIN, Gdays, SAND, DMRESP, c1=0.000358):
    """
    Calculate microbial respiration rate (MRESPt) for year t (gC/m²/year).

    Parameters:
        RAIN (float): Mean Annual Precipitation (MAP) for year t (mm/year).
        Gdays (float): Total number of days in the growing season.
        SAND (float): Sand percentage in the top 30 cm of soil.
        DMRESP (float): Daily microbial respiration (gC/m²/year).
        c1 (float): Coefficient 1 from microbial respiration rate linear equation. Default = 0.000358.

    Returns:
        float: Microbial respiration rate (MRESPt) for year t.
    """
    WETDAYS = (c1 * RAIN - 0.025) * Gdays
    MRESPt = WETDAYS * (0.7 + 0.3 * SAND / 100) * DMRESP
    return MRESPt



def calc_Se(Sk, Edays, S0=None, r=0.05):
    """
    Calculate biomass at the start of the grazing episode (Se).

    Parameters:
        Sk (float): Steady-state biomass in the absence of grazing.
        Edays (float): Number of days within the growing season prior to the grazing episode.
        S0 (float, optional): Initial biomass condition prior to the growing season. Default is 0.1 * Sk.
        r (float, optional): Relative growth rate of grass biomass. Default is 0.05.

    Returns:
        float: Biomass at the start of the grazing episode (Se).
    """
    if S0 is None:
        S0 = 0.1 * Sk

    Se = (Sk * S0) / (Sk * np.exp(-r * Edays) + S0 * (1 - np.exp(-r * Edays)))
    return Se




def calc_Lg(Ddays, d, n, W=None, Cg=None):
    """
    Calculate the biomass loss during the grazing episode.

    Parameters:
        Ddays (float): Number of days in the grazing episode.
        d (float): Stocking density (head/ha).
        n (float): Number of "pastures" per total area, A.
        W (float, optional): Average animal body size (kg live weight). Default is None.
        Cg (float, optional): Daily consumption rate (g/animal/day). Default is None.

    Returns:
        float: Biomass loss during the grazing episode (Lg).
    """
    if Cg is not None:
        Cg = Cg
    elif W is not None:
        Cg = 2 * (5300 + 770 * math.log(W))
    else:
        raise ValueError("You need to either provide Cg (daily consumption) or W (animal body size).")

    Lg = Ddays * d * Cg * n * 10**-4
    return Lg



def calc_Sg(Sk, Se, Ddays, n, d, r, W=None, Cg=None):
    """
    Calculate biomass at the end of the grazing episode (Sg).

    Parameters:
        Sk (float): Steady-state biomass in the absence of grazing.
        Se (float): Biomass at the start of the grazing episode (output of calc_Se).
        Ddays (float): Number of days of the grazing episode.
        n (float): Number of "pastures" per total area, A.
        d (float): Stocking density (head/ha).
        r (float): Relative growth rate of grass biomass.
        W (float, optional): Average animal body size (kg live weight). Default is None.
        Cg (float, optional): Daily consumption rate (g/animal/day). Default is None.

    Returns:
        float: Biomass at the end of the grazing episode (Sg).
    """
    if Cg is not None:
        Cg = Cg
    elif W is not None:
        Cg = 2 * (5300 + 770 * np.log(W))
    else:
        raise ValueError("You need to either provide Cg (daily consumption) or W (animal body size).")

    g = (d * Cg * n * 10**(-4)) / Se
    Sg = (Sk * Se) / (Sk * np.exp(-(r - g) * Ddays) + Se * (1 - np.exp(-(r - g) * Ddays)))
    
    return Sg


def calc_Sf(Sk, Sg, r, Fdays):
    """
    Calculate biomass at the end of the growing season (Sf).

    Parameters:
        Sk (float): Steady-state biomass in the absence of grazing.
        Sg (float): Biomass at the end of the grazing episode (output of calc_Sg).
        r (float): Relative growth rate of grass biomass.
        Fdays (float): Number of days left in the growing season after the grazing episode.

    Returns:
        float: Biomass at the end of the growing season (Sf).
    """
    Sf = (Sk * Sg) / (Sk * np.exp(-(r * Fdays)) + Sg * (1 - np.exp(-(r * Fdays))))
    return Sf


def calc_Pg(Se, Sg, Sf, Sk, S0=None):
    """
    Calculate the total seasonal grass production (Pg).

    Parameters:
        Se (float): Biomass at the start of the grazing episode (output of calc_Se).
        Sg (float): Biomass at the end of the grazing episode (output of calc_Sg).
        Sf (float): Biomass at the end of the growing season (output of calc_Sf).
        Sk (float): Steady-state biomass in the absence of grazing.
        S0 (float, optional): Initial biomass condition prior to the growing season. Default is 0.1 * Sk.

    Returns:
        float: Total seasonal grass production (Pg).
    """
    if S0 is None:
        S0 = 0.1 * Sk

    Pg = (Se - S0) + (Sf - min(Sg, Se))
    return Pg


def calc_Lo(Cg, Gdays, d):
    """
    Calculate the loss of biomass during the off-season.

    Parameters:
        Cg (float): Daily consumption rate (g/animal/day). Assumes half this rate during the off-season.
        Gdays (float): Total number of days in the growing season.
        d (float): Stocking density (head/ha).

    Returns:
        float: Biomass loss during the dormant season (Lo).
    """
    Lo = (Cg / 2) * (365 - Gdays) * d * 10**-4
    return Lo




def calc_PDSOCt(BNPPt_est, Sf, Lo, LIGCELL, FIRE):
    """
    Calculate Plant-Derived SOC (PDSOCt) for year t.

    Parameters:
        BNPPt_est (float): Belowground Net Primary Production estimate (output of calc_BNPPest).
        Sf (float): Biomass at the end of the growing season (output of calc_Sf).
        Lo (float): Loss of biomass during the off-season (output of calc_Lo).
        LIGCELL (float): Lignin and cellulose content of livestock feed for year t (%).
        FIRE (float): Average number of fires per year (#/year).

    Returns:
        float: Plant-Derived SOC (PDSOCt) for year t.
    """
    PDSOCt = 0.45 * ((LIGCELL * (Sf - Lo / 2) * (1 - FIRE)) + (LIGCELL + 0.05) * BNPPt_est)
    return PDSOCt

def calc_DDSOCt(LIGCELL, Ddays, Cg, n, d, Lo):
    """
    Calculate dung-derived SOC (DDSOC) in year t.

    Parameters:
    - LIGCELL (float): Lignin and cellulose content of livestock feed for year t (%).
    - Ddays (int): Number of days in the grazing episode.
    - Cg (float): Daily consumption rate (g/animal/day). For the dormant season, the rate is assumed to be half this value.
    - n (int): Number of "pastures" per total area.
    - d (float): Stocking density (head/ha).
    - Lo (float): Loss of biomass during the off-season (g/m^2).

    Returns:
    - DDSOCt (float): Dung-derived soil organic carbon (gC/m^2/year).
    """
    DDSOCt = LIGCELL * 0.45 * 0.3 * ((Ddays * Cg * n * d * 10**(-4)) + Lo)
    return DDSOCt


def calc_dmax(Sf, Sk, Cg, Gdays):
    """
    Calculate the theoretical maximum sustainable stocking density.

    Parameters:
    - Sf (float): Biomass at the end of the growing season (g/m^2).
    - Sk (float): Steady-state biomass in the absence of grazing (g/m^2).
    - Cg (float): Daily consumption rate (g/animal/day).
    - Gdays (int): Total number of days in the growing season.

    Returns:
    - dmax (float): Maximum sustainable stocking density (animals/ha).
    """
    dmax = ((Sf - 0.1 * Sk) * 10**4) / ((Cg / 2) * (365 - Gdays))
    return dmax


def calc_SOCeq(PDSOCt, DDSOCt, SAND, RAIN, Gdays, lowSOC=False, c1=0.000358):
    """
    Calculate Equilibrium SOC (SOCeq).
    
    Parameters:
        PDSOCt (float): Output of calc_PDSOCt().
        DDSOCt (float): Output of calc_DDSOCt().
        SAND (float): Sand percentage in the top 30 cm soil.
        RAIN (float): Mean annual precipitation for the year (mm/year).
        Gdays (float): Total number of days in the growing season.
        lowSOC (bool): Whether lowSOC regression equation is used. Default is False.
        c1 (float): Coefficient for microbial respiration rate. Default is 0.000358.
    
    Returns:
        float: Equilibrium SOC (SOCeq).
    """
    WETDAYS = (c1 * RAIN - 0.025) * Gdays

    if lowSOC:
        SOCeq = ((PDSOCt + DDSOCt) / (WETDAYS * (0.7 + 0.3 * (SAND / 100)) * np.exp(-10.872))) ** (1 / 1.296)
    else:
        SOCeq = ((PDSOCt + DDSOCt + 0.579 * WETDAYS * (0.7 + 0.3 * (SAND / 100))) /
                 (c1 * WETDAYS * (0.7 + 0.3 * (SAND / 100)))) + 0.579

    return SOCeq


def calc_dmax(Sf, Sk, Cg, Gdays):
    """
    Calculate the theoretical maximum sustainable stocking density.

    Parameters:
    - Sf (float): Biomass at the end of the growing season (g/m^2).
    - Sk (float): Steady-state biomass in the absence of grazing (g/m^2).
    - Cg (float): Daily consumption rate (g/animal/day).
    - Gdays (int): Total number of days in the growing season.

    Returns:
    - dmax (float): Maximum sustainable stocking density (animals/ha).
    """
    dmax = ((Sf - 0.1 * Sk) * 10**4) / ((Cg / 2) * (365 - Gdays))
    return dmax

In [2]:
def SNAPGRAZE(SAND, RAIN, MAT, FIRE, LIGCELL, 
              Sk=None, S0=None, Edays=None, Ddays=None, Fdays=None, Gdays=None, 
              d=None, n=None, W=None, Cg=None, r=0.05, APCcorrection=False, lowSOC=False, DEPTH=30):
    """
    SNAPGRAZE wrapper function to calculate SOCeq.
    """

    # Gdays calculation
    if Gdays is None:
        Gdays = 22.99 * MAT - 0.94 * MAT**2 + 0.073 * RAIN

    # Sk calculation
    if Sk is None:
        Sk = calc_ANPPmax(RAIN, MAT,SAND) / 0.9

    # S0 default
    if S0 is None:
        S0 = 0.1 * Sk

    # Cg calculation
    if Cg is None:
        Cg = 2 * (5300 + 770 * np.log(W))

    # Episodic Herbivory Model (EHM)
    Se = calc_Se(Sk, Edays, S0, r)
    Lg = calc_Lg(Ddays, d, n, W, Cg)
    Sg = calc_Sg(Sk, Se, Ddays, n, d, r, W, Cg)
    Sf = calc_Sf(Sk, Sg, r, Fdays)
    Pg = calc_Pg(Se, Sg, Sf, Sk, S0)
    Lo = calc_Lo(Cg, Gdays, d)

    # Productivity
    ANPPt_max = calc_ANPPmax(RAIN, MAT, SAND)
    ANPPt_est = calc_ANPPest(Se, Sg, Sf, Sk, S0)
    BNPPt_est = calc_BNPPest(RAIN, MAT, ANPPt_est, Sk, S0, APCcorrection, DEPTH)

    # SOC
    PDSOCt = calc_PDSOCt(BNPPt_est, Sf, Lo, LIGCELL, FIRE)
    DDSOCt = calc_DDSOCt(LIGCELL, Ddays, Cg, n, d, Lo)

    SOCeq = calc_SOCeq(PDSOCt, DDSOCt, SAND, RAIN, Gdays, lowSOC)

    return SOCeq


In [20]:
SNAPGRAZE(SAND=10,RAIN=1000,MAT=10,FIRE=2,LIGCELL=0.359,W=400,
          Edays=40,Ddays=20,Fdays=60, r = 0.05,n=4,d=2)

40.31408921668027